<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-30 16:06:20
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.32 C
-------------------
Today PnL: -17.36 K (-0.13%)
Current PnL: -16.36 L (-11.4%)
CY Booked + Current PnL: -11.88 L (-8.28%)
-------------------
Total profit:  4.02 L
Total loss:  -20.38 L
-------------------
Total Booked + Current PnL: 15.10 L (12.95%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.39%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.51 L (60.21%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.41 C (106.62%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-3.65,H-LC,92.47,236704.0,42012.0,7077.0,1.06,21.58,2.99,25.21,29.0,5.94,1.85,30.89,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,86.02,101729.0,23367.0,10661.0,-0.40,29.82,10.48,43.43,71.0,2.19,0.80,36.06,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,4.23,X-LC,31.18,154878.0,14069.0,11461.0,0.22,9.99,7.40,18.13,17.0,1.23,1.21,22.29,XY25,NTT,REFINERIES
68,SBIN,760.30,863.0,-12.51,M-LC,48.39,215260.0,13780.0,13432.0,1.88,6.84,6.24,13.51,73.0,1.03,1.68,20.30,XY25,NTT,BANKS
56,PIDILITIND,2504.06,3576.0,-15.90,X-LC,35.48,93216.0,18094.0,14066.0,2.56,24.09,15.09,42.81,11.0,1.29,0.73,16.90,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.0,6716.67,M-SC,98.92,74802.0,-19008.0,95627.0,12.71,-20.26,127.84,81.67,203.0,-0.20,0.59,44.86,XR,NTT,CERAMICS
23,DEN,52.79,75.0,115.03,L-SC,82.80,60576.0,-23888.0,59425.0,7.04,-28.28,98.10,42.07,235.0,-0.40,0.47,28.64,AR,NTT,ENTERTAINMENT
9,AWL,342.88,485.0,-67.73,H-MC,40.86,174625.0,-43104.0,133344.0,4.19,-19.80,76.36,41.45,86.0,-0.32,1.37,16.65,XY24,NTT,FMCG
85,UNIONBANK,123.87,163.0,-4.91,M-LC,83.87,166900.0,26060.0,18426.0,4.03,18.50,11.04,31.59,76.0,1.41,1.31,44.98,XY24,NTT,BANKS
45,JPPOWER,18.73,26.2,-41.93,L-SC,55.91,117684.0,-24158.0,80731.0,2.78,-17.03,68.60,39.88,236.0,-0.30,0.92,19.26,XY24,NTT,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,855.34,M-SC,16.13,129938.0,-51205.0,96674.0,-4.83,-28.27,74.40,25.10,204.0,-0.53,1.02,4.00,XY24,NTT,HEALTHCARE
15,BSOFT,628.85,844.81,-25.64,M-SC,37.63,70088.0,-39961.0,77756.0,-4.81,-36.31,110.94,34.34,171.0,-0.51,0.55,15.38,XR,ATH,IT
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,21.51,61459.0,-52090.0,92139.0,-3.44,-45.87,149.92,35.27,238.0,-0.57,0.48,50.42,XR,NTT,HOTELS
58,QUESS,711.66,986.00,NaN,NaN,5.38,68711.0,-87143.0,147220.0,-2.59,-55.91,214.26,38.55,173.0,-0.59,0.54,5.21,XY24,NTT,MISC
71,SIEMENS,4406.90,7969.85,-9.69,X-LC,75.27,81640.0,-28532.0,117602.0,-2.50,-25.90,144.05,80.85,5.0,-0.24,0.64,20.24,AR,ATH,ELECTRICAL


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.78,X-LC,51.08,248469.0,17870.0,113252.0,-0.06,7.75,45.58,56.86,1.0,0.16,1.94,15.00,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.12,X-LC,24.73,238975.0,-26058.0,105890.0,-0.99,-9.83,44.31,30.12,2.0,-0.25,1.87,7.18,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.11,X-LC,70.97,182480.0,-17830.0,76623.0,-1.16,-8.90,41.99,29.35,4.0,-0.23,1.43,26.07,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-9.69,X-LC,75.27,81640.0,-28532.0,117602.0,-2.50,-25.90,144.05,80.85,5.0,-0.24,0.64,20.24,AR,ATH,ELECTRICAL
32,HAVELLS,1588.02,2069.16,-7.88,X-LC,8.60,238212.0,-9519.0,84565.0,-1.53,-3.84,35.50,30.30,6.0,-0.11,1.86,8.94,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,11.68,H-SC,97.85,129106.0,31460.0,16216.0,0.50,32.22,12.56,48.82,138.0,1.94,1.01,43.43,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,86.02,101729.0,23367.0,10661.0,-0.40,29.82,10.48,43.43,71.0,2.19,0.80,36.06,XY24,NTT,MISC
49,LICI,785.05,983.0,-3.65,H-LC,92.47,236704.0,42012.0,7077.0,1.06,21.58,2.99,25.21,29.0,5.94,1.85,30.89,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3576.0,-15.90,X-LC,35.48,93216.0,18094.0,14066.0,2.56,24.09,15.09,42.81,11.0,1.29,0.73,16.90,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,35.26,M-SC,56.99,140524.0,2946.0,91467.0,-2.39,2.14,65.09,68.62,182.0,0.03,1.10,21.83,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,11.68,H-SC,97.85,129106.0,31460.0,16216.0,0.50,32.22,12.56,48.82,138.0,1.94,1.01,43.43,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,8.65,H-SC,84.95,160191.0,22331.0,15426.0,1.15,16.20,9.63,27.39,118.0,1.45,1.25,34.03,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,7.21,H-MC,59.14,101847.0,16746.0,26358.0,-0.19,19.68,25.88,50.65,92.0,0.64,0.80,28.03,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,1.42,M-LC,86.02,101729.0,23367.0,10661.0,-0.40,29.82,10.48,43.43,71.0,2.19,0.80,36.06,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-1.33,H-SC,33.33,110911.0,8907.0,36113.0,-0.29,8.73,32.56,44.14,111.0,0.25,0.87,26.39,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-29.44,M-MC,52.69,138768.0,9616.0,77446.0,2.29,7.45,55.81,67.42,155.0,0.12,1.09,34.96,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-9.71,M-LC,43.01,154795.0,3850.0,105601.0,-0.19,2.55,68.22,72.51,69.0,0.04,1.21,16.55,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,30.11,123622.0,286.0,136676.0,0.28,0.23,110.56,111.05,108.0,0.00,0.97,21.13,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,76.34,150826.0,-87.0,60542.0,-1.09,-0.06,40.14,40.06,89.0,-0.00,1.18,23.37,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-57.75,L-SC,21.51,61459.0,-52090.0,92139.0,-3.44,-45.87,149.92,35.27,238.0,-0.57,0.48,50.42,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-22.31,H-SC,64.52,88350.0,-9630.0,31497.0,0.99,-9.83,35.65,22.32,130.0,-0.31,0.69,28.14,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.42,M-LC,86.02,101729.0,23367.0,10661.0,-0.40,29.82,10.48,43.43,71.0,2.19,0.80,36.06,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,34.19,M-SC,29.03,116181.0,-26298.0,45822.0,-1.19,-18.46,39.44,13.70,217.0,-0.57,0.91,37.13,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-378.85,H-SC,100.00,118508.0,-14778.0,99369.0,-0.11,-11.09,83.85,63.47,131.0,-0.15,0.93,98.65,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-378.85,H-SC,100.00,118508.0,-14778.0,99369.0,-0.11,-11.09,83.85,63.47,131.0,-0.15,0.93,98.65,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,6716.67,M-SC,98.92,74802.0,-19008.0,95627.0,12.71,-20.26,127.84,81.67,203.0,-0.20,0.59,44.86,XR,NTT,CERAMICS
55,PFIZER,4245.50,6318.30,11.68,H-SC,97.85,129106.0,31460.0,16216.0,0.50,32.22,12.56,48.82,138.0,1.94,1.01,43.43,X40,ATH,PHARMA
77,TANLA,917.30,1963.11,-34.19,H-SC,96.77,161703.0,-77712.0,350669.0,2.22,-32.46,216.86,114.01,110.0,-0.22,1.27,48.31,AR,ATH,IT
74,STARHEALTH,551.56,761.00,6.12,M-SC,95.70,173387.0,-26829.0,102853.0,-0.23,-13.40,59.32,37.97,194.0,-0.26,1.36,39.22,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.85
1,25,41.22
2,50,70.11


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.14
XY25    14.38
X40     14.36
X40N    13.50
AR      12.46
XR      10.59
X200     1.43
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.54
M-SC    17.83
X-LC    17.34
H-MC     8.84
M-LC     7.85
H-LC     7.27
X-MC     5.60
M-MC     5.57
X-SC     2.80
L-SC     2.37
L-LC     1.15
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.42,-0.85,32.59
IT,9.96,-17.83,82.22
BANKS,9.82,-11.64,62.96
MISC,6.44,-31.46,89.60
FINANCE,5.94,-25.72,85.42
INSURANCE,5.47,4.54,24.06
PAINTS,5.45,-18.21,53.19
ELECTRICAL,4.99,-5.31,56.94
AUTO,4.46,-9.52,57.37


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867670.0,26
AR,1258077.0,12
X40N,1225424.0,17
XR,1098832.0,14
X40,764055.0,11
XY25,461650.0,10
SR,198492.0,2
X200,76623.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2266065.0,21
M-SC,2038174.0,21
X-LC,802179.0,12
M-MC,556202.0,5
H-MC,499204.0,8
X-SC,350363.0,4
X-MC,315423.0,5
M-LC,302943.0,6
L-SC,293895.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      940047.0      6
M-SC       XY24      747882.0      6
H-SC       AR        546875.0      3
           X40N      501871.0      6
M-SC       X40N      479536.0      6
X-LC       X40       453157.0      5
M-MC       XY24      397901.0      3
M-SC       XR        288526.0      4
H-LC       AR        271923.0      3
H-SC       XR        245630.0      4
M-SC       X40       221082.0      2
M-LC       XY24      183910.0      4
H-MC       XY24      168762.0      2
L-SC       XR        153739.0      2
X-LC       AR        144237.0      2
X-MC       XY24      143560.0      2
X-LC       XY25      128162.0      4
X-SC       X40N      121846.0      2
M-SC       XY25      121129.0      1
X-SC       XR        115824.0      1
           SR        112693.0      1
H-MC       XR        112066.0      1
M-LC       XR        105601.0      1
X-MC       X40N       95813.0      2
M-SC       AR         94220.0      1
           SR         85799.0      1
M-MC       AR         80855.0      1
L-SC       XY24       80731.0      1
M-MC       XR         77446.0      1
X-LC       X200       76623.0      1
X-MC       XY25       76050.0      1
H-MC       XY25       73302.0      1
           AR         60542.0      1
L-SC       AR         59425.0      1
H-MC       X40        58174.0      2
L-LC       XY25       42498.0      1
H-SC       X40        31642.0      2
H-MC       X40N       26358.0      1
M-LC       XY25       13432.0      1
H-LC       XY25        7077.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 59.0 seconds
